# Machine Learning Foundation

## Course 4, Part e: Non-Negative Matrix Factorization DEMO

This exercise illustrates usage of Non-negative Matrix factorization and covers techniques related to sparse matrices and some basic work with Natural Langauge Processing.  We will use NMF to look at the top words for given topics.

## Data

We'll be using the BBC dataset. These are articles collected from 5 different topics, with the data pre-processed. 

These data are available in the data folder (or online [here](http://mlg.ucd.ie/files/datasets/bbc.zip)). The data consists of a few files. The steps we'll be following are:

* *bbc.terms* is just a list of words 
* *bbc.docs* is a list of artcles listed by topic.

At a high level, we're going to 

1. Turn the `bbc.mtx` file into a sparse matrix (a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html) format can be useful for matrices with many values that are 0, and save space by storing the position and values of non-zero elements).
1. Decompose that sparse matrix using NMF.
1. Use the resulting components of NMF to analyze the topics that result.

## Data Setup

In [1]:
with open('data/bbc.mtx') as f:
    content = f.readlines()

In [2]:
content

['%%MatrixMarket matrix coordinate real general\n',
 '9635 2225 286774\n',
 '1 1 1.0\n',
 '1 7 2.0\n',
 '1 11 1.0\n',
 '1 14 1.0\n',
 '1 15 2.0\n',
 '1 19 2.0\n',
 '1 21 1.0\n',
 '1 29 1.0\n',
 '1 30 1.0\n',
 '1 33 1.0\n',
 '1 35 1.0\n',
 '1 41 1.0\n',
 '1 45 1.0\n',
 '1 47 2.0\n',
 '1 50 1.0\n',
 '1 52 1.0\n',
 '1 53 3.0\n',
 '1 55 1.0\n',
 '1 61 1.0\n',
 '1 62 1.0\n',
 '1 63 1.0\n',
 '1 65 1.0\n',
 '1 69 1.0\n',
 '1 80 1.0\n',
 '1 81 1.0\n',
 '1 86 1.0\n',
 '1 93 1.0\n',
 '1 99 1.0\n',
 '1 100 1.0\n',
 '1 104 1.0\n',
 '1 105 1.0\n',
 '1 106 1.0\n',
 '1 112 1.0\n',
 '1 116 3.0\n',
 '1 117 1.0\n',
 '1 118 2.0\n',
 '1 120 1.0\n',
 '1 121 1.0\n',
 '1 126 1.0\n',
 '1 131 4.0\n',
 '1 133 1.0\n',
 '1 134 3.0\n',
 '1 138 2.0\n',
 '1 140 2.0\n',
 '1 143 1.0\n',
 '1 151 2.0\n',
 '1 152 2.0\n',
 '1 175 1.0\n',
 '1 176 1.0\n',
 '1 177 1.0\n',
 '1 180 1.0\n',
 '1 184 1.0\n',
 '1 189 1.0\n',
 '1 194 1.0\n',
 '1 195 1.0\n',
 '1 198 1.0\n',
 '1 201 1.0\n',
 '1 206 1.0\n',
 '1 207 3.0\n',
 '1 208 1.0

In [3]:
content.pop(0)
content.pop(0)

'9635 2225 286774\n'

## Part 1

Here, we will turn this into a list of tuples representing a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html). Remember the description of the file from above:

* *bbc.mtx* is a list: first column is **wordID**, second is **articleID** and the third is the number of times that word appeared in that article.

So, if word 1 appears in article 3, 2 times, one element of our list will be:

`(1, 3, 2)`

In [4]:
(c.split() for c in content)

<generator object <genexpr> at 0x000002C798AE4120>

In [5]:
sparsemat = [tuple(map(int,map(float,c.split()))) for c in content]
# Let's examine the first few elements
sparsemat[:8]

[(1, 1, 1),
 (1, 7, 2),
 (1, 11, 1),
 (1, 14, 1),
 (1, 15, 2),
 (1, 19, 2),
 (1, 21, 1),
 (1, 29, 1)]

## Part 2: Preparing Sparse Matrix data for NMF 

We will use the [coo matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) function to turn the sparse matrix into an array. 

In [22]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix 

#subtracting one from each for rows and columns as each value starts at 1 and Python starts at 0  
rows = [x[1]-1 for x in sparsemat]
cols = [x[0]-1 for x in sparsemat]
values = [x[2] for x in sparsemat]
coo = coo_matrix((values, (rows, cols)))

In [23]:
with open('data/bbc.terms') as f:
    content = f.readlines()
    words = [c.split()[0] for c in content]

In [24]:
words

['ad',
 'sale',
 'boost',
 'time',
 'warner',
 'profit',
 'quarterli',
 'media',
 'giant',
 'jump',
 '76',
 '113bn',
 'three',
 'month',
 'decemb',
 'firm',
 'on',
 'biggest',
 'investor',
 'googl',
 'benefit',
 'highspe',
 'internet',
 'connect',
 'higher',
 'advert',
 'fourth',
 'quarter',
 'rose',
 'buoi',
 'oneoff',
 'gain',
 'offset',
 'dip',
 'bro',
 'user',
 'aol',
 'fridai',
 'own',
 'busi',
 'mix',
 'fortun',
 'lost',
 '000',
 'subscrib',
 'lower',
 'preced',
 'compani',
 'underli',
 'befor',
 'except',
 'item',
 'back',
 'stronger',
 'advertis',
 'revenu',
 'hope',
 'increas',
 'offer',
 'onlin',
 'servic',
 'free',
 'custom',
 'try',
 'sign',
 'exist',
 'broadband',
 'restat',
 '2000',
 '2003',
 'result',
 'follow',
 'probe',
 'secur',
 'exchang',
 'commiss',
 'sec',
 'close',
 'conclud',
 'slightli',
 'better',
 'analyst',
 'expect',
 'film',
 'divis',
 'saw',
 'slump',
 '27',
 'help',
 'boxoffic',
 'flop',
 'alexand',
 'sharp',
 'contrast',
 'third',
 'final',
 'lord',
 'r

In [25]:
with open('data/bbc.docs') as f:
    content = f.readlines()
    docs = [c.split()[0] for c in content]

In [26]:
docs

['business.001',
 'business.002',
 'business.003',
 'business.004',
 'business.005',
 'business.006',
 'business.007',
 'business.008',
 'business.009',
 'business.010',
 'business.011',
 'business.012',
 'business.013',
 'business.014',
 'business.015',
 'business.016',
 'business.017',
 'business.018',
 'business.019',
 'business.020',
 'business.021',
 'business.022',
 'business.023',
 'business.024',
 'business.025',
 'business.026',
 'business.027',
 'business.028',
 'business.029',
 'business.030',
 'business.031',
 'business.032',
 'business.033',
 'business.034',
 'business.035',
 'business.036',
 'business.037',
 'business.038',
 'business.039',
 'business.040',
 'business.041',
 'business.042',
 'business.043',
 'business.044',
 'business.045',
 'business.046',
 'business.047',
 'business.048',
 'business.049',
 'business.050',
 'business.051',
 'business.052',
 'business.053',
 'business.054',
 'business.055',
 'business.056',
 'business.057',
 'business.058',
 'business.059

In [27]:
pd.DataFrame(coo.toarray(),columns = words, index = docs)

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,Â£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
business.001,1,5,2,3,4,10,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
business.002,0,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
business.003,0,4,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
business.004,0,1,0,0,0,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
business.005,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tech.397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tech.398,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
tech.399,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tech.400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## NMF

NMF is a way of decomposing a matrix of documents and words so that one of the matrices can be interpreted as the "loadings" or "weights" of each word on a topic. 

Check out [the NMF documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) and the [examples of topic extraction using NMF and LDA](http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html).

## Part 3

Here, we will import `NMF`, define a model object with 5 components, and `fit_transform` the data created above.

In [28]:
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=818)
doc_topic = model.fit_transform(coo)

doc_topic.shape
# we should have 2225 observations (articles) and five latent features

(2225, 5)

In [30]:
# find feature with highest value per doc
np.argmax(doc_topic, axis=1)

array([1, 1, 1, ..., 3, 3, 2], dtype=int64)

## Part 4: 

Check out the `components` of this model:

In [29]:
model.components_.shape

(5, 9635)

This is five rows, each of which is a "topic" containing the weights of each word on that topic. The exercise is to _get a list of the top 10 words for each topic_. We can just store this in a list of lists.

**Note:** Just like we read in the data above, we'll have to read in the words from the `bbc.terms` file.

In [33]:
topic_word = pd.DataFrame(model.components_.round(3),
                          index = ["topic_1","topic_2","topic_3","topic_4","topic_5"],
                          columns = words)
topic_word

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,Â£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
topic_1,1.110,0.000,0.148,1.805,0.000,0.000,0.000,0.290,0.000,0.008,...,0.000,0.00,0.004,0.005,0.000,0.000,0.000,0.000,0.000,0.000
topic_2,0.907,2.135,0.556,1.620,0.015,1.233,0.091,0.059,0.347,0.206,...,0.001,0.00,0.000,0.000,0.000,0.000,0.002,0.002,0.000,0.001
topic_3,0.678,0.412,0.051,4.003,0.038,0.000,0.000,0.190,0.084,0.128,...,0.001,0.00,0.000,0.000,0.000,0.000,0.000,0.002,0.005,0.000
topic_4,0.696,0.416,0.055,1.220,0.052,0.051,0.008,1.221,0.280,0.000,...,0.025,0.02,0.061,0.075,0.136,0.021,0.040,0.029,0.028,0.020
topic_5,0.475,0.373,0.124,0.986,0.022,0.042,0.000,0.052,0.021,0.252,...,0.000,0.00,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000


The original data had 5 topics, as listed in bbc.docs (which these topic words relate to).

Business

Entertainment

Politics

Sport

Tech

In [36]:
topic_doc = pd.DataFrame(doc_topic.round(5),
                         index = (i.split(".")[0] for i in docs),
                         columns = ["topic_1","topic_2","topic_3","topic_4","topic_5"])
topic_doc

,topic_1,topic_2,topic_3,topic_4,topic_5
business,0.00000,0.27757,0.00680,0.08380,0.00000
business,0.03033,0.31001,0.00311,0.00000,0.00000
business,0.03685,0.17981,0.00000,0.00000,0.00000
business,0.00000,0.40197,0.00000,0.00000,0.00982
business,0.00061,0.14514,0.00890,0.03643,0.00765
...,...,...,...,...,...
tech,0.00000,0.10331,0.00000,0.37516,0.00000
tech,0.02170,0.05943,0.00000,0.29394,0.00000
tech,0.25508,0.22328,0.03813,0.26931,0.00000
tech,0.05870,0.09831,0.00509,0.15505,0.00511


In [37]:
topic_doc.reset_index().groupby('index').mean().idxmax()

topic_1         politics
topic_2         business
topic_3            sport
topic_4             tech
topic_5    entertainment
dtype: object

In [38]:
topic_doc.reset_index().groupby('index').mean() #.idxmax()

,topic_1,topic_2,topic_3,topic_4,topic_5
index,,,,,
business,0.023584,0.251565,0.007547,0.024401,0.006144
entertainment,0.020826,0.057754,0.036762,0.039328,0.154993
politics,0.261662,0.078559,0.014331,0.025091,0.009294
sport,0.026781,0.045077,0.167854,0.005876,0.025784
tech,0.023849,0.057403,0.115039,0.327589,0.020415


In [39]:
topic_word.T.sort_values(by='topic_1',ascending=False)

,topic_1,topic_2,topic_3,topic_4,topic_5
parti,7.272,0.000,0.000,0.000,0.0
labour,6.860,0.000,0.000,0.000,0.0
govern,6.403,2.868,0.000,0.000,0.0
elect,5.984,0.000,0.000,0.000,0.0
blair,5.031,0.000,0.000,0.000,0.0
...,...,...,...,...,...
tidal,0.000,0.017,0.000,0.000,0.0
serg,0.000,0.015,0.019,0.000,0.0
hanov,0.000,0.004,0.000,0.029,0.0
marin,0.000,0.000,0.022,0.114,0.0


In [40]:
with open('data/bbc.terms') as f:
    content = f.readlines()
words = [c.split()[0] for c in content]

In [41]:
topic_words = []
for r in model.components_:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:12]
    topic_words.append([words[e[1]] for e in a])

In [42]:
# Here, each set of words relates to the corresponding topic (ie the first set of words relates to topic 'Business', etc.)
topic_words[:5]

[['parti',
  'labour',
  'govern',
  'elect',
  'blair',
  'peopl',
  'tori',
  'minist',
  'plan',
  'brown',
  'sai',
  'told'],
 ['year',
  'increas',
  'wage',
  'compani',
  'busi',
  'minimum',
  'govern',
  'rate',
  'market',
  'economi',
  'pai',
  'rise'],
 ['game',
  'plai',
  'time',
  'player',
  'world',
  'first',
  'win',
  'get',
  'go',
  'on',
  'england',
  'two'],
 ['peopl',
  'mobil',
  'phone',
  'technolog',
  'servic',
  'music',
  'digit',
  'user',
  'network',
  'on',
  'tv',
  'system'],
 ['best',
  'song',
  'music',
  'year',
  'award',
  'film',
  '25',
  'angel',
  'robbi',
  'british',
  'think',
  'vote']]

The original data had 5 topics, as listed in `bbc.docs` (which these topic words relate to). 

```
Business
Entertainment
Politics
Sport
Tech
```

In "real life", we would have found a way to use these to inform the model. But for this little demo, we can just compare the recovered topics to the original ones. And they seem to match reasonably well. The order is different, which is to be expected in this kind of model.

In [43]:
with open('data/bbc.docs') as d:
    doc_content = d.readlines()
    
doc_content[:8]

['business.001\n',
 'business.002\n',
 'business.003\n',
 'business.004\n',
 'business.005\n',
 'business.006\n',
 'business.007\n',
 'business.008\n']

---
### Machine Learning Foundation (C) 2020 IBM Corporation